Process the pre-processed CSV files to create time series **per hard disk**

* we start capture from `D-2` (`D` being day of failure) to `NUM_DAYS_RECORD` ago
* this means that with `NUM_DAYS_RECORD` of data, we know 2 days before the drive fails
* we do this for **all failure cases** and a portion of non-failures cases

At the end, create two CSV files.

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import glob
import re

data_files = glob.glob("./processed/*.csv")
data_files.sort()
data_files = data_files[::-1]
print(len(data_files), "Files:")
print("From:", data_files[0], ", to:", data_files[-1])

1277 Files:
From: ./processed/2018-09-30.csv , to: ./processed/2015-01-01.csv


In [2]:
# loads all the dataframes

loaded_df = []

for n,data_file in tqdm_notebook(enumerate(data_files), total=len(data_files)):
    df = pd.read_csv(data_file)
    df = df.drop("failure", axis=1)
    df.set_index("serial_number", inplace=True)
    loaded_df.append(df)

In [3]:
def get_smart_data(index, serial_number):
    df = loaded_df[index]
    smart_data = list(df.loc[serial_number])[4:]
    return smart_data

In [4]:
fail_record = {}
record = {}

In [5]:
NUM_DAYS_RECORD = 7

for n,data_file in tqdm_notebook(enumerate(data_files), total=len(data_files)):
    print("File:", data_file)
    try:
        df = pd.read_csv(data_file)
        serial_numbers = list(df["serial_number"])
        failures = list(df["failure"])
        df.set_index("serial_number", inplace=True)
        interval = int(len(failures)/40)-1
        for i,failure in enumerate(failures):
            # failure
            if int(failure)==1:
                try:
                    serial_number = serial_numbers[i]
                    row = df.loc[serial_number]
                    hdd_model = int(re.sub("\D", "", row["model"]))
                    hdd_size = row["capacity_bytes"]
                    data = [hdd_model, hdd_size]
                    for d in range(1,1+NUM_DAYS_RECORD):
                        smart_data = get_smart_data(n+d, serial_number)
                        data += smart_data
                    fail_record[serial_number] = data
                except Exception as e:
                    print(e)

            # ok
            elif i%interval==0:
                if int(failure)==0:
                    try:
                        serial_number = serial_numbers[i]
                        row = df.loc[serial_number]
                        hdd_model = int(re.sub("\D", "", row["model"]))
                        hdd_size = row["capacity_bytes"]
                        data = [hdd_model, hdd_size]
                        for d in range(1,1+NUM_DAYS_RECORD):
                            smart_data = get_smart_data(n+d, serial_number)
                            data += smart_data
                        record[serial_number] = data
                    except Exception as e:
                        print(e)
    except Exception as e:
        print("file failed", e)

File: ./processed/2018-09-30.csv
File: ./processed/2018-09-29.csv
'the label [ZJV04SZ3] is not in the [index]'
'the label [ZCH0BET9] is not in the [index]'
File: ./processed/2018-09-28.csv
File: ./processed/2018-09-27.csv
'the label [ZA13C001] is not in the [index]'
File: ./processed/2018-09-26.csv
File: ./processed/2018-09-25.csv
File: ./processed/2018-09-24.csv
File: ./processed/2018-09-23.csv
File: ./processed/2018-09-22.csv
File: ./processed/2018-09-21.csv
File: ./processed/2018-09-20.csv
File: ./processed/2018-09-19.csv
File: ./processed/2018-09-18.csv
'the label [ZCH07Z30] is not in the [index]'
File: ./processed/2018-09-17.csv
File: ./processed/2018-09-16.csv
File: ./processed/2018-09-15.csv
File: ./processed/2018-09-14.csv
File: ./processed/2018-09-13.csv
File: ./processed/2018-09-12.csv
File: ./processed/2018-09-11.csv
File: ./processed/2018-09-10.csv
File: ./processed/2018-09-09.csv
File: ./processed/2018-09-08.csv
File: ./processed/2018-09-07.csv
File: ./processed/2018-09-06

'the label [ZCH0ACED] is not in the [index]'
File: ./processed/2018-04-30.csv
File: ./processed/2018-04-29.csv
File: ./processed/2018-04-28.csv
File: ./processed/2018-04-27.csv
File: ./processed/2018-04-26.csv
File: ./processed/2018-04-25.csv
File: ./processed/2018-04-24.csv
File: ./processed/2018-04-23.csv
'the label [ZA14LWDQ] is not in the [index]'
File: ./processed/2018-04-22.csv
File: ./processed/2018-04-21.csv
File: ./processed/2018-04-20.csv
File: ./processed/2018-04-19.csv
File: ./processed/2018-04-18.csv
'the label [ZCH0D398] is not in the [index]'
File: ./processed/2018-04-17.csv
'the label [ZCH0D398] is not in the [index]'
File: ./processed/2018-04-16.csv
'the label [ZCH0D398] is not in the [index]'
File: ./processed/2018-04-15.csv
File: ./processed/2018-04-14.csv
'the label [ZJV03WEA] is not in the [index]'
File: ./processed/2018-04-13.csv
File: ./processed/2018-04-12.csv
File: ./processed/2018-04-11.csv
File: ./processed/2018-04-10.csv
File: ./processed/2018-04-09.csv
File

'the label [PL1331LAGS58GH] is not in the [index]'
'the label [ZCH08192] is not in the [index]'
'the label [ZA16DDTZ] is not in the [index]'
'the label [ZCH06V7F] is not in the [index]'
File: ./processed/2017-12-21.csv
File: ./processed/2017-12-20.csv
File: ./processed/2017-12-19.csv
File: ./processed/2017-12-18.csv
File: ./processed/2017-12-17.csv
File: ./processed/2017-12-16.csv
'the label [ZA189HS7] is not in the [index]'
File: ./processed/2017-12-15.csv
File: ./processed/2017-12-14.csv
File: ./processed/2017-12-13.csv
File: ./processed/2017-12-12.csv
'the label [ZA1261SY] is not in the [index]'
'the label [ZA161S54] is not in the [index]'
File: ./processed/2017-12-11.csv
'the label [ZA13Z5S6] is not in the [index]'
File: ./processed/2017-12-10.csv
File: ./processed/2017-12-09.csv
File: ./processed/2017-12-08.csv
File: ./processed/2017-12-07.csv
'the label [ZA180TWS] is not in the [index]'
File: ./processed/2017-12-06.csv
'the label [ZA1814EX] is not in the [index]'
File: ./processe

'the label [Z305B2QN] is not in the [index]'
'the label [ZCH089J2] is not in the [index]'
'the label [ZA14SVE3] is not in the [index]'
'the label [ZA12KZ4Z] is not in the [index]'
'the label [PL1331LAHD0A0H] is not in the [index]'
'the label [PL2331LAHBAXGJ] is not in the [index]'
'the label [PL2331LAHDNNGJ] is not in the [index]'
'the label [ZA143CY2] is not in the [index]'
'the label [ZA12KYXW] is not in the [index]'
'the label [PL1331LAGSMMBH] is not in the [index]'
'the label [ZCH065NZ] is not in the [index]'
'the label [Z300ZS8K] is not in the [index]'
'the label [Z305DJ2E] is not in the [index]'
'the label [ZA16X6YQ] is not in the [index]'
'the label [S300XSJ8] is not in the [index]'
'the label [Z305D4SA] is not in the [index]'
'the label [Z305B3ZL] is not in the [index]'
'the label [ZA18BM18] is not in the [index]'
'the label [Z3015CRG] is not in the [index]'
'the label [Z304KBX9] is not in the [index]'
'the label [ZA1814EY] is not in the [index]'
'the label [ZCH06MC7] is not in

File: ./processed/2017-08-15.csv
File: ./processed/2017-08-14.csv
'the label [PL2331LAG93HNJ] is not in the [index]'
File: ./processed/2017-08-13.csv
'the label [PL1331LAGA0EUH] is not in the [index]'
File: ./processed/2017-08-12.csv
File: ./processed/2017-08-11.csv
File: ./processed/2017-08-10.csv
'the label [ZA181A4Q] is not in the [index]'
File: ./processed/2017-08-09.csv
'the label [W4J1GN19] is not in the [index]'
'the label [ZA181A4Q] is not in the [index]'
File: ./processed/2017-08-08.csv
'the label [ZA1815AY] is not in the [index]'
'the label [ZA180R4W] is not in the [index]'
File: ./processed/2017-08-07.csv
File: ./processed/2017-08-06.csv
'the label [ZA1815AY] is not in the [index]'
'the label [ZA180RYZ] is not in the [index]'
File: ./processed/2017-08-05.csv
'the label [ZA1815AY] is not in the [index]'
File: ./processed/2017-08-04.csv
'the label [ZA1815AY] is not in the [index]'
'the label [ZA180RYZ] is not in the [index]'
'the label [ZA181A4N] is not in the [index]'
File: .

...

'the label [MJ0351YNG9Z0XA] is not in the [index]'
'the label [Z302AKW7] is not in the [index]'
'the label [S301NHPX] is not in the [index]'
'the label [Z303Y422] is not in the [index]'
'the label [Z3015BTV] is not in the [index]'
'the label [Z300GPBH] is not in the [index]'
'the label [JK11A8BFG736PF] is not in the [index]'
'the label [Z300ZRZQ] is not in the [index]'
'the label [W3005RZY] is not in the [index]'
'the label [Z4D069CV] is not in the [index]'
'the label [JK11A8B9GPG2UF] is not in the [index]'
'the label [MJ1311YNG7LRMA] is not in the [index]'
'the label [PL1331LAGSH79H] is not in the [index]'
'the label [PL1331LAGT9UUH] is not in the [index]'
'the label [Z3029AQW] is not in the [index]'
'the label [PL1331LAGLWUKH] is not in the [index]'
'the label [MK0301YHGM4NLA] is not in the [index]'
'the label [PL2331LAGMSUVJ] is not in the [index]'
'the label [Z304JW2W] is not in the [index]'
'the label [Z3014A1R] is not in the [index]'
'the label [MJ0351YNGABYXA] is not in the [ind

'the label [MJ0351YNG9Z0XA] is not in the [index]'
'the label [MJ0351YNG9YEGA] is not in the [index]'
'the label [Z30228HY] is not in the [index]'
'the label [Z3025LPE] is not in the [index]'
'the label [Z4D03PPN] is not in the [index]'
'the label [MJ1311YNG36DGA] is not in the [index]'
'the label [W30077F8] is not in the [index]'
'the label [Z3021PBN] is not in the [index]'
'the label [Z300GZ4W] is not in the [index]'
'the label [S3010BL9] is not in the [index]'
'the label [Z30273GC] is not in the [index]'
'the label [MJ1311YNG36GMA] is not in the [index]'
'the label [PL1321LAG3664H] is not in the [index]'
'the label [S300Z43Q] is not in the [index]'
'the label [Z4D05LLQ] is not in the [index]'
'the label [Z300WN1M] is not in the [index]'
'the label [Z30268L9] is not in the [index]'
'the label [Z3029FR3] is not in the [index]'
'the label [Z304JKP3] is not in the [index]'
'the label [S300ZAGX] is not in the [index]'
'the label [S30117M9] is not in the [index]'
'the label [PL2331LAGN1B2

File: ./processed/2015-09-20.csv
'the label [S301GM66] is not in the [index]'
File: ./processed/2015-09-19.csv
'the label [Z303Y3BA] is not in the [index]'
File: ./processed/2015-09-18.csv
File: ./processed/2015-09-17.csv
'the label [Z303Y3BA] is not in the [index]'
File: ./processed/2015-09-16.csv
File: ./processed/2015-09-15.csv
File: ./processed/2015-09-14.csv
File: ./processed/2015-09-13.csv
File: ./processed/2015-09-12.csv
'the label [S2ZYJ9CG108692] is not in the [index]'
File: ./processed/2015-09-11.csv
File: ./processed/2015-09-10.csv
File: ./processed/2015-09-09.csv
File: ./processed/2015-09-08.csv
File: ./processed/2015-09-07.csv
File: ./processed/2015-09-06.csv
File: ./processed/2015-09-05.csv
File: ./processed/2015-09-04.csv
'the label [Z302T6CZ] is not in the [index]'
'the label [S300YQ8S] is not in the [index]'
'the label [S300YT82] is not in the [index]'
File: ./processed/2015-09-03.csv
File: ./processed/2015-09-02.csv
'the label [Z302AYRC] is not in the [index]'
'the la

File: ./processed/2015-04-23.csv
'the label [WD-WCAU42451607] is not in the [index]'
'the label [9XW06KY6] is not in the [index]'
File: ./processed/2015-04-22.csv
'the label [Z4D0A6E9] is not in the [index]'
File: ./processed/2015-04-21.csv
'the label [Z4D0ACNT] is not in the [index]'
'the label [Z4D02HXT] is not in the [index]'
'the label [Z4D05VEG] is not in the [index]'
File: ./processed/2015-04-20.csv
'the label [WD-WCC4N0298257] is not in the [index]'
File: ./processed/2015-04-19.csv
'the label [Z4D04XRY] is not in the [index]'
File: ./processed/2015-04-18.csv
'the label [Z4D04XRY] is not in the [index]'
File: ./processed/2015-04-17.csv
'the label [Z4D04XRY] is not in the [index]'
'the label [Z4D02HXT] is not in the [index]'
'the label [Z4D08L4P] is not in the [index]'
'the label [Z4D05VV1] is not in the [index]'
File: ./processed/2015-04-16.csv
'the label [WD-WCAU44995264] is not in the [index]'
'the label [6XW05FKB] is not in the [index]'
'the label [WD-WCAU45414650] is not in t

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
'the label [PL1331LAGSA6LH] is not in the [index]'
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index ou

In [6]:
print(len(fail_record[list(fail_record.keys())[0]]), "length:\n", fail_record[list(fail_record.keys())[0]])

317 length:
 [80000055, 8001563222016, 35257528, 0.0, 0, 11, 0, 2963033381, 0.0, 13348, 0, 0.0, 11, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 34360262668.0, 0.0, 31.0, 4296.0, 7.0, 12164.0, 31, 35257528.0, 0.0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13332.0, 45420224712.0, 95865785556.0, 0.0, 0.0, 0.0, 0.0, 0.0, 107111848, 0.0, 0, 11, 0, 2958041911, 0.0, 13325, 0, 0.0, 11, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 34360262668.0, 0.0, 32.0, 4291.0, 7.0, 12112.0, 32, 107111848.0, 0.0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13308.0, 45379128328.0, 95734062516.0, 0.0, 0.0, 0.0, 0.0, 0.0, 167836786, 0.0, 0, 11, 0, 2952445755, 0.0, 13301, 0, 0.0, 11, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 34360262668.0, 0.0, 32.0, 4287.0, 7.0, 12078.0, 32, 167836786.0, 0.0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 13284.0, 45342011976.0, 95587600460.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8546056, 0.0, 0, 9, 0, 2946107190, 0.0, 13278, 0, 0.0, 9, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 34360262668.0, 0.0, 31.0, 4281.0, 7.0, 12034.0, 31

In [7]:
print(len(record[list(record.keys())[0]]), "length:\n", record[list(record.keys())[0]])

317 length:
 [4000000, 4000787030016, 51937688, 0.0, 0, 10, 0, 635089820, 0.0, 14608, 0, 0.0, 10, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 22.0, 0.0, 0.0, 34005.0, 22, 0.0, 0.0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14359.0, 36668380824.0, 33531373166.0, 0.0, 0.0, 0.0, 0.0, 0.0, 88752888, 0.0, 0, 10, 0, 633680208, 0.0, 14584, 0, 0.0, 10, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 22.0, 0.0, 0.0, 34005.0, 22, 0.0, 0.0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14336.0, 36652382824.0, 33340019739.0, 0.0, 0.0, 0.0, 0.0, 0.0, 145630168, 0.0, 0, 10, 0, 632451534, 0.0, 14560, 0, 0.0, 10, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 22.0, 0.0, 0.0, 34005.0, 22, 0.0, 0.0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14312.0, 36637078000.0, 33199384991.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9051704, 0.0, 0, 10, 0, 630881864, 0.0, 14536, 0, 0.0, 10, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 22.0, 0.0, 0.0, 34005.0, 22, 0.0, 0.0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14288.0, 366207776

In [8]:
header_row = ["failure", "hdd_model", "hdd_capacity"] + ["feat"+str(i) for i in range(len(fail_record[list(fail_record.keys())[0]])-2)]

In [9]:
assert len(header_row) == len(fail_record[list(fail_record.keys())[0]])+1
assert len(header_row) == len(record[list(record.keys())[0]])+1

Replace previously generated CSV files

In [10]:
import csv

with open('train.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(header_row)
    for key, value in fail_record.items():
        writer.writerow([1]+value)
    for key, value in record.items():
        writer.writerow([0]+value)